In [1]:
import pandas as pd
df = pd.read_csv('amazon_data-4.csv')

In [2]:
df

,Product_ID,product,price,ratings,reviews,url,category
0,065fbe37-c7d5-4a49-a104-be663cc0d662,"SAMSUNG Galaxy S23 Ultra Cell Phone, Factory U...",197.50,4.7,2711,https://www.amazon.com/Xiaomi-Unlocked-Warrant...,Smartphones
1,7edaf615-2474-40db-87d2-01ca4538256b,"SAMSUNG Galaxy S23 Ultra Cell Phone, Factory U...",999.99,4.2,597,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
2,48048dcd-3bf6-454a-a6a6-50acd5f8538a,SAMSUNG Galaxy S9+ Factory Unlocked Smartphone...,349.99,4.1,8484,https://www.amazon.com/Samsung-Galaxy-S9-Unloc...,Smartphones
3,9b37bb66-4551-469a-99ef-565f889f095a,"SAMSUNG Galaxy S23 Ultra Cell Phone, Factory U...",999.99,4.7,377,https://www.amazon.com/SAMSUNG-Factory-Unlocke...,Smartphones
4,45d073fb-ab85-42d9-91b8-c1e7c062e273,Google Pixel 7a - Unlocked Android Cell Phone ...,444.00,4.5,518,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
...,...,...,...,...,...,...,...
282,0a6b9ce0-179e-42b2-a4ad-a546a4830601,"Grisofaa Clothes Organizer for Folded Clothes,...",49.99,4.5,84,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
283,9366c64b-30a0-4497-ab26-4d0badb95d91,ULENDIS 6 Pack Upgraded Washable Wardrobe Clot...,25.99,4.0,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
284,feca223c-f7e9-46d6-be9f-65874d308fc3,ULENDIS 9 Pack Upgraded Wardrobe Clothes Organ...,36.99,4.5,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
285,07f2589c-7dc7-4a94-b586-8f583aed5c4a,Womens Skinny Stretch Cinch Belt Elastic Waist...,11.99,5.0,452,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories


In [8]:
df['price'].duplicated().sum()

139

In [ ]:
import pandas as pd
import psycopg2
import re
# Load the JSON data into a pandas DataFrame
df = pd.read_json('amazon_data_cat.json')
# Remove any duplicates that may have been created due to URL changes
df = df.drop_duplicates(subset=['Product_ID'], keep='first')
# Fill NaN values in 'reviews' with 0
df['reviews'].fillna(0, inplace=True)
# Check the data type of the reviews column
if pd.api.types.is_string_dtype(df['reviews']):
    df['reviews'] = df['reviews'].str.replace('(', '').str.replace(')', '')
# Convert reviews to integer
df['reviews'] = df['reviews'].astype(int)
# Set negative reviews to 0
df.loc[df['reviews'] < 0, 'reviews'] = 0
# Fill NaN values in 'url' with 'Unknown'
df['url'].fillna('Unknown', inplace=True)
# Function to extract the second occurrence of the URL
def extract_second_url(url):
    prefix = "https://www.amazon.comhttps://"
    if url.startswith(prefix):
        matches = re.findall(r'https://www\.amazon\.com/', url[len(prefix):])
        if len(matches) >= 1:
            second_occurrence_index = url.rfind(matches[0])
            return url[second_occurrence_index:]
    return url
# Apply the function to the 'url' column
df['url'] = df['url'].apply(extract_second_url)
# Remove any duplicates that may have been created due to URL changes
df = df.drop_duplicates(subset=['product', 'price','ratings', 'reviews', 'category'], keep='first')
# Replace empty product names with NaN and drop those rows
df['product'].replace('', pd.NA, inplace=True)
df.dropna(subset=['product'], inplace=True)
# Drop the 'review_responders' column
df.drop('review_responders', axis=1, inplace=True)
# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="demopass"
)
cur = conn.cursor()
# Create a table in the PostgreSQL database
create_table_query = """
DROP TABLE IF EXISTS amazon_data;
CREATE TABLE IF NOT EXISTS amazon_data (
    product_id TEXT NOT NULL,
    product TEXT NOT NULL,
    price NUMERIC NOT NULL,
    ratings NUMERIC NOT NULL,
    reviews INTEGER NOT NULL,
    category TEXT NOT NULL,
    url TEXT NOT NULL
)
"""
cur.execute(create_table_query)
conn.commit()
def clean_format_data(row):
    # Convert the ratings value to a float
    ratings = float(row['ratings'])
    # Convert the product name to a string and then adapt for SQL insertion
    product = psycopg2.extensions.adapt(str(row['product']).encode('utf-8')).getquoted().decode('utf-8')[1:-1]
    # Convert price to float, if not possible set to 0
    try:
        price = float(row['price'])
    except ValueError:
        price = 0
    category = psycopg2.extensions.adapt(row['category']).getquoted().decode('utf-8')[1:-1]
    url = row['url']
    product_id = row['Product_ID']
    reviews = row['reviews']  # Already cleaned and converted to int
    return product_id, product, price, ratings, reviews, category, url
# Insert the data from the pandas DataFrame into the PostgreSQL table
for index, row in df.iterrows():
    product_id, product, price, ratings, reviews, category, url = clean_format_data(row)
    insert_query = """
    INSERT INTO amazon_data (product_id, product, price, ratings, reviews, category, url)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    cur.execute(insert_query, (product_id, product, price, ratings, reviews, category, url))
conn.commit()
cur.close()
conn.close()
# Save the DataFrame to a CSV file
df.to_csv('amazon_data.csv', index=False)